# Generate images using a pretrained Model

In [1]:
import torch
import matplotlib.pyplot as plt

from torchvision.utils import make_grid

from diffusers import DDPMPipeline

## Hyperparameters

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

image_size = (64, 64)
nc = 3

num_inference_steps = 1000

model_path = './data/weights/diffusion.pth'

## Initialization

In [ ]:
# load checkpoint
module = torch.load(model_path, map_location=device)
module.eval().to(device)

# pipeline
model = module.model

pipeline = DDPMPipeline(model, module.noise_scheduler)
generator = torch.Generator(device=device).manual_seed(0)

## Image generation

### Grid

In [ ]:
# grid size (6x6), change n to 1 for single image generation
n = 6

images = pipeline(n*n, generator, num_inference_steps, "numpy").images

grid = make_grid(torch.from_numpy(images.transpose(0, 3, 1, 2)), nrow=n, normalize=True).swapaxes(0, 2).swapaxes(0, 1)

plt.imshow(grid.detach().cpu().numpy())
plt.axis('off')